In [1]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
import numpy as np
from Utils.utils import create_URM, create_ICM, combine_matrices, create_submission, write_submission
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Evaluation.K_FoldEvaluator import K_FoldEvaluator
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeTwoModelsByScores, MergeThreeModelsByScores

In [3]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()

URM_train_list=[]
URM_validation_list=[]
evaluator_list=[]

In [4]:
for k in range(5):
    URM_train,URM_validation = split_train_in_two_percentage_global_sample(URM, train_percentage=0.8)
    URM_train_list.append(URM_train)
    URM_validation_list.append(URM_validation)
    
    ##SELECT 25% OF USERS WITH LONGEST PROFILES TO FINE-TUNE THE HYPERPARAMETERS
    users_to_ignore=[]
    profile_length = np.ediff1d(URM_train.indptr)
    block_size = int(len(profile_length) * 0.25)

    start_pos = 3 * block_size
    end_pos = min(4 * block_size, len(profile_length))
    sorted_users = np.argsort(profile_length)

    users_in_group = sorted_users[start_pos:end_pos]

    users_in_group_p_len = profile_length[users_in_group]
    sorted_users = np.argsort(profile_length)

    users_not_in_group_flag = np.isin(sorted_users, users_in_group, invert=True)
    evaluator_list.append(EvaluatorHoldout(URM_validation, cutoff_list=[10],verbose=False,ignore_users=sorted_users[users_not_in_group_flag]))

In [6]:
URM_combined_list=[]
for URM_i in URM_train_list:
    URM_combined_list.append(combine_matrices(ICM=ICM,URM=URM_i))

In [7]:
from bayes_opt import BayesianOptimization

In [8]:
rp3_recommenders=[]
slime_recommenders=[]
similarity_recommenders=[]


for i in range(len(URM_train_list)):
    
    rp3_recommenders.append(RP3betaRecommender(URM_train=URM_combined_list[i]))
    rp3_recommenders[i].fit(topK=190, 
                            alpha=0.6481627715453979, 
                            beta=0.4870075599959749, 
                            implicit=True, 
                            normalize_similarity=True)
    
    slime_recommenders.append(MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_combined_list[i]))
    slime_recommenders[i].fit(topK=501, 
                              l1_ratio=0.0003245363418031065, 
                              alpha=0.07706408849201826, 
                              workers = 20)
    
    sim_slime = slime_recommenders[i].W_sparse
    sim_rp3 = rp3_recommenders[i].W_sparse
    
    similarity_recommenders.append(ItemKNNSimilarityHybridRecommender(URM_train=URM_combined_list[i],
                                                                     Similarity_1=sim_rp3,
                                                                     Similarity_2=sim_slime))
    


100%|████████████████████████████████████▉| 18056/18059 [10:09<00:00, 29.60it/s]

 26%|█████████▉                            | 4744/18059 [02:40<03:56, 56.35it/s]


 46%|█████████████████▌                    | 8328/18059 [04:36<19:36,  8.27it/s]


 67%|████████████████████████▉            | 12168/18059 [06:34<09:06, 10.77it/s]


 87%|████████████████████████████████▏    | 15720/18059 [08:20<00:27, 83.73it/s]


  7%|██▊                                   | 1352/18059 [01:06<26:01, 10.70it/s]


100%|████████████████████████████████████▉| 18056/18059 [11:52<00:00, 25.34it/s]

 31%|███████████▋                          | 5576/18059 [03:22<03:08, 66.36it/s]


 51%|███████████████████▎                  | 9192/18059 [05:35<03:11, 46.35it/s]


 71%|██████████████████████████▎          | 12872/18059 [07:42<05:14, 16.50it/s]


 92%|██████████████████████████████████   | 16648/18059 [09:46<01:55, 12.27it/s]


100%|████████████████████████████████████▉| 18032/18059 [10:02<00:00, 79.81it/s]

In [9]:
tuning_params = {
    "alpha":(0.1,0.25),
    "topK":(480,530)
}

In [10]:

def BO_function(alpha,topK):
    MAP = 0
    k = 5
    cutoff=10
    for i in range(len(similarity_recommenders)):
        similarity_recommenders[i].fit(topK=int(topK),alpha=alpha)
        result_df, _ = evaluator_list[i].evaluateRecommender(similarity_recommenders[i])
        MAP += result_df.loc[cutoff]["MAP"]
        
    return MAP/k


In [11]:
optimizer = BayesianOptimization(f=BO_function,
                                pbounds=tuning_params,
                                verbose=5,
                                random_state=5)

optimizer.maximize(init_points=100,n_iter=100)


|   iter    |  target   |   alpha   |   topK    |
-------------------------------------------------


100%|████████████████████████████████████▉| 18056/18059 [10:13<00:00, 79.81it/s]

|  1        |  0.2904   |  0.1333   |  523.5    |
|  2        |  0.2904   |  0.131    |  525.9    |


100%|████████████████████████████████████▉| 18056/18059 [11:34<00:00, 26.00it/s]


|  3        |  0.2904   |  0.1733   |  510.6    |
|  4        |  0.29     |  0.2149   |  505.9    |
|  5        |  0.2903   |  0.1445   |  489.4    |
|  6        |  0.2903   |  0.1121   |  516.9    |
|  7        |  0.2903   |  0.1662   |  487.9    |
|  8        |  0.2901   |  0.232    |  493.7    |
|  9        |  0.2904   |  0.1621   |  494.8    |
|  10       |  0.2901   |  0.1943   |  509.0    |
|  11       |  0.2904   |  0.19     |  493.3    |
|  12       |  0.2903   |  0.1427   |  492.7    |
|  13       |  0.2903   |  0.1491   |  487.2    |
|  14       |  0.2903   |  0.1248   |  528.2    |
|  15       |  0.2898   |  0.244    |  489.4    |
|  16       |  0.2903   |  0.1036   |  490.2    |
|  17       |  0.2904   |  0.205    |  519.0    |
|  18       |  0.2903   |  0.1034   |  508.9    |
|  19       |  0.2903   |  0.1002   |  505.8    |
|  20       |  0.2902   |  0.196    |  529.3    |
|  21       |  0.2904   |  0.1389   |  520.1    |
|  22       |  0.2903   |  0.2306   |  526.1    |


|  103      |  0.2902   |  0.1138   |  529.0    |
|  104      |  0.2905   |  0.1888   |  519.8    |
|  105      |  0.2905   |  0.1657   |  524.9    |
|  106      |  0.2905   |  0.175    |  519.8    |
|  107      |  0.2905   |  0.1705   |  527.0    |
|  108      |  0.2902   |  0.2051   |  520.1    |
|  109      |  0.2904   |  0.2471   |  527.8    |
|  110      |  0.2904   |  0.1759   |  524.9    |
|  111      |  0.2905   |  0.1403   |  524.6    |
|  112      |  0.2903   |  0.1903   |  510.3    |
|  113      |  0.2905   |  0.1628   |  519.8    |
|  114      |  0.2904   |  0.1881   |  510.2    |
|  115      |  0.2901   |  0.2054   |  502.3    |
|  116      |  0.2906   |  0.1496   |  524.9    |
|  117      |  0.2905   |  0.1742   |  527.0    |
|  118      |  0.2906   |  0.148    |  524.6    |
|  119      |  0.2904   |  0.1352   |  524.9    |
|  120      |  0.2906   |  0.1624   |  524.6    |
|  121      |  0.2906   |  0.18     |  519.8    |
|  122      |  0.2905   |  0.177    |  524.6    |


In [13]:
import json
with open("tuning_KFOLD/similarity_merging_SLIM_RP3_BEST25/" + similarity_recommenders[0].RECOMMENDER_NAME + "_logs.json", 'w') as json_file:
    json.dump(optimizer.max, json_file)